In [11]:
import read__geolife as rg
import os, time
import pandas as pd

In [12]:
def del_traj_day(df, col_name):
    d = pd.DataFrame(df[col_name].value_counts())
    d.columns = ['nums']
    # 少于5天的次的全部删除
    d = d[d['nums'] < 5]
    delindexs = d.index
    print(len(delindexs))
    return df[~df[col_name].isin(delindexs)]

In [13]:
def get_ts_mode_mean(df):
    t1 = df.groupby(['lat', 'lon']).size().reset_index()
    t1.columns = ['a', 'b', 'count']
    t2 = t1.groupby(['a'])['count'].max().reset_index()
    t2.columns = ['a', 'max_count']
    t1 = t1.merge(t2, on=['a'], how='left')
    t1 = t1[t1['count']==t1['max_count']]
    t1 = t1.groupby(['a'])['b'].mean().reset_index()
    t1

In [14]:
# 每個timeslot的軌跡是最後一次出現的軌跡
def get_slot_by_unix(ts):
    dt = time.localtime(ts)
    day, hour, minute = dt.tm_mday, dt.tm_hour, dt.tm_min
    slot = hour * 2 + (0 if minute <= 30 else 1)
    return slot

In [15]:
def ts_to_slot(ts):
    ans = [0] * 48
    for t in ts:
        slot = get_slot_by_unix(t)
        ans[slot] = 1
    return ans

In [16]:
def traj_to_slot(trajectory, ts, pad=0):
    ans = [pad] * 48
    for i in range(len(ts)):
        slot_id = get_slot_by_unix(ts[i])
        ans[slot_id] = trajectory[i]
    return ans

In [17]:
def get_loc_index(lon, lat):
    df = pd.read_csv('./loc_index.csv')


In [18]:
# all users
df_all = rg.read_all_users('../data/geolife/Data')
df_all.sort_values(by=['time'],inplace=True)

[1/182] processing user 169
[2/182] processing user 054
[3/182] processing user 107
[4/182] processing user 171


/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0
/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0


[5/182] processing user 098
[6/182] processing user 013
[7/182] processing user 007
[8/182] processing user 089
[9/182] processing user 165


/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0


[10/182] processing user 102
[11/182] processing user 006
[12/182] processing user 126


...

[164/182] processing user 032
[165/182] processing user 074
[166/182] processing user 034
[167/182] processing user 116
[168/182] processing user 137
[169/182] processing user 105
[170/182] processing user 046
[171/182] processing user 071


/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0
/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0


[172/182] processing user 153


/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0
/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0
/home/lizhenyu/Project/E2DTC/preprocess/read__geolife.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0


[173/182] processing user 156
[174/182] processing user 056
[175/182] processing user 021
[176/182] processing user 157
[177/182] processing user 024
[178/182] processing user 008
[179/182] processing user 130
[180/182] processing user 181
[181/182] processing user 164
[182/182] processing user 066


In [21]:
df_all
df_all.to_hdf('./Geolife_all_user.h5',key='data')

In [92]:
# 删除非北京的采样点
lon = [115.416827, 117.508251]
lat = [39.442078, 41.058964]
df = df_all[(df_all.lat>lat[0])&(df_all.lat<lat[1])&(df_all.lon>lon[0]) & (df_all.lon<lon[1])]
df

,time,lat,lon,alt,label,user
0,2000-01-01 23:12:19,39.988992,116.327023,128.937005,0,163
1,2000-01-01 23:13:21,39.990964,116.327041,221.128615,0,163
2,2000-01-01 23:15:23,39.993207,116.326827,217.191591,0,163
0,2007-04-12 09:31:32,39.974233,116.330383,823.490814,0,142
1,2007-04-12 09:39:37,39.974317,116.330450,823.490814,0,142
...,...,...,...,...,...,...
291,2012-07-27 08:31:03,39.979481,116.303499,131.760207,0,163
292,2012-07-27 08:31:05,39.979475,116.303497,131.097241,0,163
293,2012-07-27 08:31:10,39.979374,116.303522,126.235679,0,163
294,2012-07-27 08:31:15,39.979190,116.303587,119.301427,0,163


In [ ]:
df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))
df['year'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_year)
df['month'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mon)
df['day'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mday)
df['slot_id'] = df['timestamp'].apply(lambda x: get_slot_by_unix(x))
df = df.drop_duplicates(subset=['user', 'year', 'month', 'day', 'slot_id'])
df.sort_values(by='user',inplace=True)
df.reset_index(drop=True, inplace=True)
df

/tmp/ipykernel_2214526/3021142272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))


In [ ]:
lon_list = (df['lon'].value_counts().index.tolist())
lon_list.sort(reverse=True)
lon_list

df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))
df['year'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_year)
df['month'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mon)
df['day'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mday)
df['slot_id'] = df['timestamp'].apply(lambda x: get_slot_by_unix(x))
df = df.drop_duplicates(subset=['user', 'year', 'month', 'day', 'slot_id'])
df.sort_values(by='user',inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
loc_index = df[['lat','lon']]
loc_index = loc_index.sort_values(by='lat').drop_duplicates().reset_index(drop=True)
loc_index.to_csv('loc_index.csv')

In [ ]:
loc_index

In [62]:
# dict:(lat,lon)--->index
loc = [tuple(x) for x in loc_index.values.tolist()]
index = loc_index.index.tolist()
loc_index_dict = dict(zip(loc, index))

tmp = df[['lat', 'lon']]
index = []
for i in range(len(tmp)):
    index.append(loc_index_dict[(tmp.iloc[i][0], tmp.iloc[i][1])])
df['loc_index'] = index
df

/tmp/ipykernel_2214526/4152561819.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc_index'] = index


,time,lat,lon,alt,label,user,timestamp,year,month,day,slot_id,loc_index
0,2000-01-01 23:12:19,39.988992,116.327023,128.937005,0,163,946768339,2000,1,2,14,44847
0,2007-04-12 09:31:32,39.974233,116.330383,823.490814,0,142,1176370292,2007,4,12,35,26376
9,2007-04-12 10:05:00,39.974517,116.330067,406.824147,0,142,1176372300,2007,4,12,36,26741
0,2007-04-12 10:18:53,39.975517,116.330283,351.049869,0,161,1176373133,2007,4,12,36,29837
0,2007-04-12 10:21:16,39.976467,116.330067,173.884514,2,163,1176373276,2007,4,12,36,31392
...,...,...,...,...,...,...,...,...,...,...,...,...
0,2012-07-25 19:03:00,39.984960,116.310739,489.429065,0,163,1343242980,2012,7,26,6,40194
316,2012-07-25 19:31:04,39.994674,116.445402,150.249462,0,163,1343244664,2012,7,26,7,48392
0,2012-07-26 08:01:06,39.991750,116.440100,489.500253,0,163,1343289666,2012,7,26,32,46901
0,2012-07-27 08:07:00,39.990677,116.444623,487.477461,0,163,1343376420,2012,7,27,32,45819


In [63]:
df['loc_index'].max()

69422

In [64]:
seqs = []
for (user_index, year, month, day), group in df.groupby(['user', 'year', 'month', 'day']):
    group.reset_index(inplace=True)
    if group.shape[0] < 12:
        continue
    ts = group['timestamp']
    seq = group['loc_index'].tolist()
    seq = traj_to_slot(seq, ts, pad='[PAD]')

    if seq.count('[PAD]') > 36:
        continue
    seq = [str(x) for x in seq]
    seqs.append([" ".join(seq), user_index, year, month, day])

    # print('uid:', user_index,'date:', month, '-', day)
    # print("group shape:", group.shape, '[PAD] count:', seq.count('[PAD]'))
    # print(group.head())

    # print(group)
    # print(ts)
    # print(seq)
    # print(seqs)

In [65]:
all_traj = pd.DataFrame(seqs, columns=['trajectory', 'user_index', 'year', 'month', 'day'])
all_traj = del_traj_day(all_traj,'user_index')
all_traj

60


,trajectory,user_index,year,month,day
0,54062 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3
2,56423 56409 56278 56398 56243 56209 56436 5626...,0,2009,4,4
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6
...,...,...,...,...,...
1890,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13
1891,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15
1892,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2
1893,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12


In [66]:
all_traj.reset_index(drop= True)

,trajectory,user_index,year,month,day
0,54062 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3
2,56423 56409 56278 56398 56243 56209 56436 5626...,0,2009,4,4
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6
...,...,...,...,...,...
1775,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13
1776,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15
1777,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2
1778,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12


In [67]:
all_traj.to_hdf('all_traj.h5', key='data')

In [327]:
# num users
len(dict(all_traj['user_index'].value_counts()))

67

In [320]:
# duration
print(all_traj['year'].max(), all_traj['year'].min())

2012 2007
